<center><img src="https://images2.mobilissimo.ro/ejM/662234d3d923f.jpg" width=400></center>

# Introduction

Llama 3 is the latest model from Meta. Let's try and see what we can do with it.  

Will test now the following model:
* **Model**: Llama3
* **Version**: 8b-chat-hf
* **Framework**: Transformers
* **Version**: V1

This is what we will test:

* Simple prompts with general information questions
* Poetry (haiku, sonets) writing
* Code writing (Python, C++, Java)
* Software design (simple problems)
* Multi-parameter questions
* Chain of reasoning
* A more complex reasoning problem

I intend to learn from this experience so that I can then build then someting a bit more complex.

Note: this notebook is organized to facilitate a comparison of **Llama3** with **Gemma**. Corresponding notebook is [Prompt Gemma like a Pro](https://www.kaggle.com/code/gpreda/prompt-gemma-like-a-pro). At the end of current notebook we are reviewing the performance of the two notebooks for the list of tasks submitted to both.

# Preparation

We import the libraries we need, and we set the model to be ready for testing.

## Import packages

In [1]:
from time import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

## Define model

This will take some time (loading checkpoint shards, cca. 2 min.).

In [2]:
model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

2024-05-13 08:31:00.615024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 08:31:00.615179: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 08:31:00.714953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Prepare query function

In [3]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
"""

## Utility function for output format

In [4]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Test with few simple geography and history questions

In [5]:
response = query_model(
    system_message,
    user_message="What is the surface temperature of the Moon?",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What is the surface temperature of the Moon? 

**<font color='green'>Answer:</font>** The surface temperature of the Moon varies greatly depending on the time of day and the location on the Moon. During the day, the temperature can reach as high as 253°F (122°C), while at night it can drop to as low as -243°F (-153°C). 

**<font color='magenta'>Total time:</font>** 6.26 sec.

Let's try also with a different prompt.

In [6]:
response = query_model(    
    system_message,
    user_message="What is the surface of France?",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What is the surface of France? 

**<font color='green'>Answer:</font>** The surface area of France is approximately 551,500 square kilometers (213,011 square miles). 

**<font color='magenta'>Total time:</font>** 1.45 sec.

It doesn't look like a well crafted prompt helps too much.  Let's get back to our initial prompt pattern for more questions.

In [7]:
response = query_model(
    system_message,
    user_message="When was the 30 years war?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** When was the 30 years war? 

**<font color='green'>Answer:</font>** The Thirty Years War was from 1618 to 1648. 

**<font color='magenta'>Total time:</font>** 1.04 sec.

In [8]:
response = query_model(
    system_message,
    user_message="What is graphe paranomon?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What is graphe paranomon? 

**<font color='green'>Answer:</font>** Graphe paranomon is a type of ancient Greek legal action, specifically a public accusation made against a magistrate or other public official for acting contrary to the laws or customs of the city-state. 

**<font color='magenta'>Total time:</font>** 2.74 sec.

In [9]:
response = query_model(
    system_message,
    user_message="Who was the next shogun after Tokugawa Ieyasu?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Who was the next shogun after Tokugawa Ieyasu? 

**<font color='green'>Answer:</font>** Tokugawa Hidetada. 

**<font color='magenta'>Total time:</font>** 0.65 sec.

In [10]:
response = query_model(
    system_message,
    user_message="What was the name of the Chinese dinasty during 1st century B.C.?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What was the name of the Chinese dinasty during 1st century B.C.? 

**<font color='green'>Answer:</font>** The Han Dynasty. 

**<font color='magenta'>Total time:</font>** 0.37 sec.

Let's try some elementary questions about American history.

In [11]:
response = query_model(
    system_message,
    user_message="Who was the first American president?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Who was the first American president? 

**<font color='green'>Answer:</font>** George Washington. 

**<font color='magenta'>Total time:</font>** 0.3 sec.

In [12]:
response = query_model(
    system_message,
    user_message="When took place the Civil War in United States of America?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** When took place the Civil War in United States of America? 

**<font color='green'>Answer:</font>** The American Civil War took place from 1861 to 1865. 

**<font color='magenta'>Total time:</font>** 1.11 sec.

# Let's write poetry

In [13]:
system_message = """
You are an AI assistant designed to write poetry.
Please answer with a haiku format (17 words poems).
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a poem about Boris Becker wins in tennis",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a poem about Boris Becker wins in tennis 

**<font color='green'>Answer:</font>** Golden racket's song
Boris Becker's mighty serve
Victory's sweet cry 

**<font color='magenta'>Total time:</font>** 1.25 sec.

In [14]:
response = query_model(
    system_message,
    user_message="Please write a poem about Shakespeare being lame at playing poker",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a poem about Shakespeare being lame at playing poker 

**<font color='green'>Answer:</font>** Bard's bluffing fails hard
Shakespeare's poker face a lie
Lame, his royal flush 

**<font color='magenta'>Total time:</font>** 1.59 sec.

In [15]:
system_message = """
You are an AI assistant designed to write poetry.
Please answer with a short poem, with rime, in the style of Shakespeare's poems.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a poem about Nadia Comaneci winning Montreal Olympiad",
    temperature=0.1, 
    max_length=512)
display(Markdown(colorize_text(response)))

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 104



**<font color='red'>Question:</font>** Please write a poem about Nadia Comaneci winning Montreal Olympiad 

**<font color='green'>Answer:</font>** Fair Nadia, with thy nimble frame,
Didst conquer Montreal's Olympic fame.
Thy gymnastics, a wondrous, whirling dance,
Didst capture hearts, and earn a second chance.

Thy score, a seven, a feat most rare,
Didst stun the crowd, and leave them bare.
Of breath, of words, of all articulate sound,
As thou, a maiden, didst thy triumph wound.

The judges' eyes, in awe, didst gaze upon
Thy form, so lithe, so strong, so young, and won.
The crowd, in raptures, didst thy name proclaim,
As thou, a heroine, didst thy glory claim.

And when the scores, in final tally, told
Thy total, a perfect ten, to behold,
The world, in wonder, didst thy feat proclaim,
And Nadia Comaneci, forever in fame. 

**<font color='magenta'>Total time:</font>** 13.14 sec.

# Math problem and Python code

In [16]:
system_message = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in Python to calculate the area of a circle of radius r",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Python to calculate the area of a circle of radius r 

**<font color='green'>Answer:</font>** Here is the Python code for the function:

```
import math

def calculate_circle_area(r):
    area = math.pi * (r ** 2)
    return area
``` 

**<font color='magenta'>Total time:</font>** 2.64 sec.

In [17]:
response = query_model(
    system_message,
    user_message="Please write a function in Python to order a list",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Python to order a list 

**<font color='green'>Answer:</font>** Here is a Python function that orders a list:

```
def order_list(lst):
    return sorted(lst)
``` 

**<font color='magenta'>Total time:</font>** 1.66 sec.

# Software design

In [18]:
response = query_model(
    system_message,
    user_message="""Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                        """,
    temperature=0.1, 
    max_length=1024)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                         

**<font color='green'>Answer:</font>** Here is the Python code for the PhoneBook class:

```
class PhoneBook:
    def __init__(self):
        self.contacts = []

    def add_contact(self, name, surname, address, phone):
        self.contacts.append({
            'name': name,
           'surname': surname,
            'address': address,
            'phone': phone
        })

    def delete_contact(self, name):
        for contact in self.contacts:
            if contact['name'] == name:
                self.contacts.remove(contact)
                print(f"Contact {name} deleted successfully.")
                return
        print(f"Contact {name} not found.")

    def order_by_name(self):
        self.contacts.sort(key=lambda x: x['name'])

    def search_contact(self, name):
        for contact in self.contacts:
            if contact['name'] == name:
                return contact
        return None

    def display_contacts(self):
        for contact in self.contacts:
            print(f"Name: {contact['name']}, Surname: {contact['surname']}, Address: {contact['address']}, Phone: {contact['phone']}")


# Example usage:
phone_book = PhoneBook()
phone_book.add_contact('John', 'Doe', '123 Main St', '1234567890')
phone_book.add_contact('Jane', 'Smith', '456 Elm St', '9876543210')
phone_book.order_by_name()
phone_book.display_contacts()
print(phone_book.search_contact('John'))
phone_book.delete_contact('John')
phone_book.display_contacts()
``` 

**<font color='magenta'>Total time:</font>** 21.97 sec.

In [19]:
response = query_model(
    system_message,
    user_message="""Please write a small Python module that creates a REST API service
                        with two endpoints: 
                        * get_status (GET)
                        * prediction (POST)
                        The prediction endpoint receives the payload, extract three fields: city, street and number
                        and returns a field called price_estimate
                        
                        """,
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a small Python module that creates a REST API service
                        with two endpoints: 
                        * get_status (GET)
                        * prediction (POST)
                        The prediction endpoint receives the payload, extract three fields: city, street and number
                        and returns a field called price_estimate
                        
                         

**<font color='green'>Answer:</font>** Here is a simple Python module using Flask to create a REST API service with the two endpoints you requested:

```
from flask import Flask, request, jsonify
import json

app = Flask(__name__)

# Sample data for prediction
data = {
    "New York": {"street": {"1st": 100, "2nd": 120, "3rd": 150}, 
                 "Los Angeles": {"1st": 80, "2nd": 90, "3rd": 110}, 
                 "Chicago": {"1st": 70, "2nd": 80, "3rd": 100}
}

@app.route('/get_status', methods=['GET'])
def get_status():
    return jsonify({'status': 'OK'})

@app.route('/prediction', methods=['POST'])
def prediction():
    data = request.get_json()
    city = data.get('city')
    street = data.get('street')
    number = data.get('number')
    
    if city and street and number:
        if city in data and street in data[city] and number in data[city][street]:
            return jsonify({'price_estimate': data[city][street][number]})
        else:
            return jsonify({'error': 'Invalid city 

**<font color='magenta'>Total time:</font>** 17.72 sec.

# C++ code

In [20]:
system_message = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in C++ to calculate the area of a circle of radius r", 
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to calculate the area of a circle of radius r 

**<font color='green'>Answer:</font>** Here is the C++ code for the function to calculate the area of a circle:

```cpp
#include<iostream>
using namespace std;

double circleArea(double r) {
    return 3.14 * r * r;
}

int main() {
    double radius;
    cout << "Enter the radius of the circle: ";
    cin >> radius;
    cout << "The area of the circle is: " << circleArea(radius);
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 6.63 sec.

In [21]:
system_message = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in C++ to calculate the volume of a cylinder with radius r and height h",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to calculate the volume of a cylinder with radius r and height h 

**<font color='green'>Answer:</font>** Here is the C++ code for the function to calculate the volume of a cylinder:

```cpp
#include<iostream>
using namespace std;

double cylinderVolume(double r, double h) {
    return 3.14159 * r * r * h;
}

int main() {
    double r, h;
    cout << "Enter the radius of the cylinder: ";
    cin >> r;
    cout << "Enter the height of the cylinder: ";
    cin >> h;
    cout << "The volume of the cylinder is: " << cylinderVolume(r, h) << endl;
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 8.75 sec.

In [22]:
response = query_model(
    system_message,
    user_message="Please write a function in C++ to order a list", 
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to order a list 

**<font color='green'>Answer:</font>** Here is a simple implementation of a function in C++ that sorts a list of integers in ascending order using the built-in `sort` function from the `<algorithm>` library:

```cpp
#include <iostream>
#include <algorithm>
#include <vector>

void sortList(std::vector<int>& list) {
    std::sort(list.begin(), list.end());
}

int main() {
    std::vector<int> list = {4, 2, 9, 6, 5, 1, 3, 7, 8};
    std::cout << "Original list: ";
    for (int i : list) {
        std::cout << i << " ";
    }
    std::cout << std::endl;

    sortList(list);

    std::cout << "Sorted list: ";
    for (int i : list) {
        std::cout << i << " ";
    }
    std::cout << std::endl;

    return 0;
}
```

This code defines a function `sortList` that takes a reference to a `std::vector<int>` as its argument. It uses the `std::sort` function to sort the list in ascending order. The `main` function demonstrates how to use this function by 

**<font color='magenta'>Total time:</font>** 17.66 sec.

# Multiple parameters questions

## Best food in France

In [23]:
system_message = """
You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
"""
user_message = """
What are the {adjective} {number} {items} from {place}?
"""
response = query_model(
    system_message,
    user_message.format(
    adjective="best",
    number="3",
    items="food",
    place="France"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best 3 food from France?
 

**<font color='green'>Answer:</font>** Bonjour! France, the land of cuisine, is renowned for its delectable dishes and exquisite culinary delights. Here are the top 3 foods from France that you must try:

• **Croissants**: Flaky, buttery, and oh-so-flavorful, croissants are a classic French breakfast staple. Whether plain or filled with chocolate, they're a must-try when visiting France.

• **Escargots**: Snails in garlic butter, anyone? Escargots are a quintessential French appetizer that will delight your taste buds. The perfect combination of rich flavors and tender texture.

• **Macarons**: These delicate meringue-based confections are a French specialty. With a wide range of flavors and fillings, macarons are the perfect treat to satisfy your sweet tooth. From classic flavors like raspberry and chocolate to more unique flavors like matcha and rose, there's a macaron for everyone.

Bon appétit! 

**<font color='magenta'>Total time:</font>** 13.54 sec.

## Best attractions in Italy

In [24]:
response = query_model(    
    system_message,
    user_message.format(
    adjective="best",
    number="five",
    items="attractions",
    place="Italy"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best five attractions from Italy?
 

**<font color='green'>Answer:</font>** Italy is a country with a rich history, art, architecture, and culture, offering numerous attractions for tourists. Here are the top 5 attractions in Italy:

• **Colosseum** (Rome): This ancient amphitheater is one of Rome's most iconic landmarks and a must-visit attraction. Learn about the history of gladiators and the Roman Empire while exploring the ancient structure.

• **Leaning Tower of Pisa** (Pisa): This famous tower is a symbol of Italy and a popular spot for tourists. Take a stroll around the square, admire the tower's unique architecture, and capture the perfect photo.

• **Cinque Terre** (Liguria): These five colorful towns on the Italian coast offer breathtaking ocean views, scenic hiking trails, and charming villages to explore. Visit Monterosso, Vernazza, Corniglia, Manarola, and Riomaggiore for a unique experience.

• **Uffizi Gallery** (Florence): This world-renowned museum is home to some of the most famous artworks in history, including Botticelli's "The Birth of Venus" and Leonardo da Vinci's "Annunciation." Discover the works of Italian masters like Michelangelo, Raphael, and Caravaggio.

 

**<font color='magenta'>Total time:</font>** 17.66 sec.

## Most affordable places to stay in Spain

In [25]:
response = query_model(
    system_message,
    user_message.format(
    adjective="most affordable",
    number="two",
    items="places to retire",
    place="Spain"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the most affordable two places to retire from Spain?
 

**<font color='green'>Answer:</font>** **Affordable Retirement Options in Spain**

Considering the cost of living, taxes, and quality of life, here are two of the most affordable places to retire from Spain:

• **Almería**: Located in the southeastern region of Andalusia, Almería is one of the most affordable cities in Spain. The cost of living is relatively low, with a monthly budget of around €1,500-€2,000 (approximately $1,700-$2,200 USD) covering accommodation, food, and other expenses.

• **Girona**: Situated in the northeastern region of Catalonia, Girona is another affordable option for retirees. The city has a lower cost of living compared to larger cities like Barcelona, with a monthly budget of around €2,000-€2,500 (approximately $2,200-$2,800 USD) covering the essentials.

Both Almería and Girona offer a great quality of life, with pleasant weather, rich cultural heritage, and a strong sense of community. Additionally, they have good access to healthcare, amenities, and transportation networks, making them attractive options for retirees looking to make the most of their retirement in Spain. 

**<font color='magenta'>Total time:</font>** 16.65 sec.

## Less known but great places to stay in Romania

In [26]:
response = query_model(    
    system_message,
    user_message.format(
    adjective="Less known but great",
    number="4",
    items="places to stay",
    place="Romania"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the Less known but great 4 places to stay from Romania?
 

**<font color='green'>Answer:</font>** Here are 4 lesser-known but great places to stay in Romania:

• **Sighisoara**: This medieval town is a UNESCO World Heritage Site and offers a unique blend of Germanic and Latin architecture. Stay in one of the many charming guesthouses or boutique hotels, such as the Hotel Casa Wagner or the Guesthouse Casa Sighisoara.

• **Sinaia**: Located in the heart of the Carpathian Mountains, Sinaia is a picturesque town surrounded by stunning scenery. Stay at the elegant Hotel Palace or the cozy Guesthouse Casa Sinaia for a relaxing getaway.

• **Târgu Mureș**: This city in central Romania is often overlooked by tourists, but it has a rich cultural heritage and a unique blend of Hungarian and Romanian influences. Stay at the Hotel Continental or the Guesthouse Casa Târgu Mureș for a comfortable and affordable stay.

• **Râșnov**: This charming village in Transylvania is famous for its medieval fortress and stunning scenery. Stay at the Guesthouse Casa Râșnov or the Hotel Râșnov for a peaceful retreat amidst the rolling hills and forests.

These places offer a unique and authentic Romanian experience, away from the crowds of popular tourist destinations like Bucharest or Bras 

**<font color='magenta'>Total time:</font>** 17.65 sec.

## Best commedies by Shakespeare

In [27]:
response = query_model(
    system_message,
    user_message.format(
    adjective="best",
    number="3",
    items="commedies",
    place="Shakespeare entire creation"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best 3 commedies from Shakespeare entire creation?
 

**<font color='green'>Answer:</font>** What a delightful question! Here are three of the most popular and enduring comedies from Shakespeare's entire creation:

• **A Midsummer Night's Dream**: A whimsical tale of love, magic, and mischief in the forest, featuring the iconic characters of Puck, Oberon, and Titania.

• **Twelfth Night**: A witty and romantic comedy that explores the complexities of love, identity, and deception, featuring the memorable characters of Viola, Olivia, and the lovable fool, Feste.

• **As You Like It**: A charming and lighthearted comedy that follows the adventures of Rosalind, a clever and resourceful heroine, as she navigates love, family, and social hierarchy in the Forest of Arden.

These three comedies showcase Shakespeare's mastery of language, character development, and comedic storytelling, and continue to delight audiences to this day! 

**<font color='magenta'>Total time:</font>** 12.52 sec.

## Most important battles from WW2

In [28]:
response = query_model(
    system_message,
    user_message.format(
    adjective="most important",
    number="5",
    items="battles",
    place="WW2"
    ), 
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the most important 5 battles from WW2?
 

**<font color='green'>Answer:</font>** Here are the top 5 most important battles of World War II:

• **Battle of Stalingrad (August 1942 - February 1943)**: A decisive turning point on the Eastern Front, where the Soviet Union defeated Nazi Germany, marking a shift in momentum in favor of the Allies.

• **Battle of Midway (June 4-7, 1942)**: A pivotal naval battle in the Pacific, where the United States defeated Japan, preventing a potential invasion of Hawaii and halting Japan's expansion in the Pacific.

• **Battle of El Alamein (October - November 1942)**: A crucial battle in North Africa, where the British Eighth Army defeated the German-Italian forces, marking a turning point in the North African Campaign and paving the way for the Allied invasion of Italy.

• **Battle of Normandy (D-Day) (June 6, 1944)**: A massive Allied invasion of Nazi-occupied France, which marked a significant turning point in the war, as it established a foothold for the Allies on the European mainland.

• **Battle of Berlin (April - May 1945)**: The final battle of the war, where Soviet forces captured the German capital, leading to the surrender of Germany and the end of World War II in Europe.

These battles had a significant impact on the outcome of the war and are widely regarded as some of the most important conflicts of World War II. 

**<font color='magenta'>Total time:</font>** 20.25 sec.

# Multiple steps reasoning (task chain)

In [29]:
system_message = """
You are an AI assistant designed to answer questions in a chain of thought.
Use the answer to the first question as input for the second question.
"""
user_message= """
Question one: What are the best city from {country} for tourists?
Question two: What are the best {number} of attractions from the city identified as answer to question one?
Answer:
"""
response = query_model(
    system_message,
    user_message.format(
    number="3",
    country="France"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
Question one: What are the best city from France for tourists?
Question two: What are the best 3 of attractions from the city identified as answer to question one?


**<font color='green'>Answer:</font>**
 

**<font color='green'>Answer:</font>** Question one: What are the best cities from France for tourists?



**<font color='green'>Answer:</font>** Paris is often considered one of the best cities for tourists in France. 

**<font color='magenta'>Total time:</font>** 2.1 sec.

# Reasoning like Einstein will do

In [30]:
system_message = """
You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.
Reasoning: Think step by step. Explain your reasoning.
Question: {question}
Answer:
"""
t = time()
response = query_model(
    system_message,
    user_message="How many kilometers will travel totally the swallow?", 
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))



**<font color='red'>Question:</font>** How many kilometers will travel totally the swallow? 

**<font color='green'>Answer:</font>** Let's break this down step by step.

1. The train travels from Bucharest to Ploiesti, which is a distance of 60 km. Since the train is moving at a constant speed of 60 km/h, it will take 1 hour to cover this distance.
2. The swallow flies from Ploiesti to the moving train, which is a distance of 60 km. Since the swallow is flying at a constant speed of 90 km/h, it will take 40 minutes (60 km / 90 km/h) to cover this distance.
3. When the swallow reaches the train, it flies back towards Ploiesti, which is also a distance of 60 km. Again, it will take 40 minutes to cover this distance.
4. The train and the swallow continue to move towards Ploiesti. Since the train is moving at 60 km/h and the swallow is flying at 90 km/h, the distance between them is decreasing at a rate of 30 km/h (90 km/h - 60 km/h).
5. The swallow flies back and forth between the train and Ploiesti until the train reaches Ploiesti. Since the train is moving at a constant speed, it will take 1 hour to cover the remaining distance.
6. During this time, the swallow will make multiple trips back and forth between the train and Ploiesti. Let's calculate the total distance the swallow will travel during this time.

The distance between the train and Ploiesti is decreasing at a rate of 30 km/h. In 1 hour, the distance will decrease by 30 km. Since the swallow is flying at 90 km/h, it will make 3 trips back and forth during this time (30 km / 30 km/h = 1 hour).

Each trip is a distance of 60 km, so the total distance the swallow will travel during this time is:

3 trips x 60 km/trip = 180 km

Adding this to the initial distance the swallow traveled to reach the train, we get:

40 minutes x 90 km/h = 36 km (initial distance) + 180 km = 216 km

So, the swallow will travel a total distance of 216 km. 

**<font color='magenta'>Total time:</font>** 32.64 sec.



**<font color='magenta'>Total time:</font>** 32.64 sec.

# Conclusions


We compared the results with those with Gemma. Here is the notebook for Gemma: [Prompt Gemma like a Pro](https://www.kaggle.com/code/gpreda/prompt-gemma-like-a-pro).


* Llama3 answered quite decent to most of the questions on history and geography. It looks like an improvement from previous runs of this Notebook.
* Llama3 is good at code generation and math problems.
* The final math problem - failed worst than Gemma (who rationed it's 120 Km; the true answer is 90 km). But actually the rationing is really, really wrong.

In terms of execution time, after I applied the change suggested in one of the comments, it improved very much. 

So, comparison of Llama3 8B with Gemma 2B is in favor of Llama3 for general knowledge, maths and code generation, with Gemma slightly better on the more complex math task.

Stay tuned, we will continue with more tests in the next days.